In [2]:
import random
random.seed(10)
import numpy as np
np.random.seed(10)

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image
import keras

In [4]:
#!pip install tensorflow

In [5]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__) 

2.18.0


checking each folder

In [6]:
print(os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1'))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\new\\Downloads\\PRCP-1001-RiceLeaf\\data1'

In [ ]:
print(os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Bacterial leaf blight'))

In [ ]:
print((os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Brown spot')))

In [ ]:
print((os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\leaf smut')))

checking the no. of images in each classes

In [ ]:
#Bacterial leaf blight
print(len(os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Bacterial leaf blight')))

In [ ]:
# Brown spot
print(len(os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Brown spot')))

In [ ]:
#Leaf smut
print(len(os.listdir(r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\leaf smut')))

here Bacterial leaf blight and Brown spot contain 40 images each, while leaf smut contains only 39 images.

visualizing the images in each classes and checking the image shape

In [ ]:
#!pip install pillow
from PIL import Image

In [ ]:
data_paths = {
    "Bacterial leaf blight": r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Bacterial leaf blight',
    "Brown spot": r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Brown spot' ,
    "Leaf smut": r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\leaf smut'
}

# Number of images to display from each path
num_images_to_display = 10

# Loop through each path and category name
for category_name, path in data_paths.items(): # Iterate through the dictionary items to get both key and value
    # Get a list of image files in the path
    image_files = [f for f in os.listdir(path) if f.endswith(('.jpg', '.png'))]



    # Display the selected images
    for image_file in image_files:
        image_path = os.path.join(path, image_file)
        img = mpimg.imread(image_path)
        print(img.shape)
        img = Image.open(image_path)

        # Get image format
        print("Image Format:", img.format)
        plt.imshow(img)
        plt.title(category_name)  # Optionally display the image filename as the title
        plt.axis('off')  # Turn off axis ticks and labels
        plt.show()

Most of the images in 'Bacterial leaf blight' are of same size ,Image shape: (897, 3081, 3).
But i can see the size vary in 'Brown spot' and ' Leaf smut'. some are of like  (88, 296, 3), (168, 359, 3),(323, 1504, 3)  and so on in Brown spot.
and in Leaf smut sizes of some images are like (73, 367, 3),(174, 503, 3) ,(897, 3081, 3). All images are of JPEG format.
So, we need to resize the images to the same size. Let's resize them to (224, 224, 3) so that the image dimensions are not too large and it doesn't take up excessive memory.


In [ ]:
CATEGORIES=['Bacterial leaf blight','Brown spot','Leaf smut']

This dataset contains 119 images of rice leaf (JPEG).  These are grouped into 3 different folders according to disease type 'Bacterial leaf blight','Brown spot','Leaf smut'.
So we are creating an array called training[] to store images that are resized to (224,224,3) and the index at which the image in the CATEGORIES list.

In [ ]:
#!pip install opencv-python

In [ ]:
import cv2
training = []  #Training is an array that will contain image pixel values and the index at which the image in the CATEGORIES list.
datapath= r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1'
def createTrainingData():  # creating function
  for category in CATEGORIES:
    path = os.path.join(datapath, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (512
                                         , 512)) # i am resizing all images to 512x512
      training.append([new_array, class_num])
createTrainingData() # calling the function

shuffling the dataset

In [ ]:
import random
random.shuffle(training)


 Assigning Labels and Features


In [ ]:
X =[]
y =[]
for features, label in training:
  X.append(features)
  y.append(label)


In [ ]:
X = np.array(X)  #  reshaping  list of images (X) into a 4D NumPy array

In [ ]:

print(X[:2])


In [ ]:
#visualizing first 10 resized images in X
plt.figure(figsize=(10,5))
plotnum=1

for i in range(0,10):
  plt.subplot(2,5,plotnum)
  plotnum+=1

  plt.imshow(X[i])
  plt.title(y[i])
  plt.axis('off')


In [ ]:
print(y)

In [ ]:
len(y)

Normalising X and OneHotEncoding on converted labels 0,1,2

In [ ]:
from tensorflow.keras.utils import to_categorical
X = X.astype('float32')
X /= 255

Y = to_categorical(y, 3)

# to_categorical is a function used to convert categorical class labels (e.g., integers like 0, 1, 2) into one-hot encoded vectors.
# For example, the class 0 would be converted to [1, 0, 0], class 1 to [0, 1, 0], etc.
# This is necessary for classification problems where each class is represented by a vector in the output layer of a neural network.

print(Y[100]) # printing 100th element in Y
print(Y.shape)

In [ ]:
print(X[:1])
print(Y[:1])

In [ ]:
print(type(Y))

#### Splitting dataset into training, validation and test ore. 

The validation set is a set of data, separate from the training set, that is used to validate our model performance during training.

This validation process gives information that helps us tune the model’s hyperparameters and configurations accordingly. It is like a critic telling us whether the training is moving in the right direction or not.

The model is trained on the training set, and, simultaneously, the model evaluation is performed on the validation set after every epoch.

The main idea of splitting the dataset into a validation set is to prevent our model from overfitting.

In [ ]:
from sklearn.model_selection import train_test_split
# split the data into training(here temp) and testing dataset. we are splitting in 80%,20% for training AND TESTING
X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [ ]:
print(len(X_temp))
print(len(X_test))

In [ ]:
# break training set into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size = 0.1, random_state = 4)

In [ ]:
print(Y_train)

In [ ]:
# print number of training, validation, and test images
print(len(X_train), 'train samples')
print(len(X_test), 'test samples')
print(len(X_val), 'validation samples')

### creating model

In [ ]:
from keras.models import Sequential #import
from tensorflow.keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential() #This initializes an empty model. You will add layers to this model one by one.

#creating 3 convolution layers
model.add(Conv2D(filters=8, kernel_size=2, padding='same', activation='relu',
                        input_shape=(512, 512, 3)))   # using 8 filters in 1st convolution layer, so the output will have 8 feature maps. Fewer filters mean fewer features are detected,
                                                      # and thus fewer parameters are learned.
                                                     # kernal_size is 2x2
                                                    # padding='same' means that the output feature map will have the same width and height as the input. 
                                                     #input shape is the shape of input image. we had resized it to (224,224,3)

model.add(MaxPooling2D(pool_size=2))  # Pooling is a downsampling operation that reduces the spatial dimensions (height and width) of the input,
                                      #typically by selecting the maximum value from a set of pixels within a small region.
                        #for each 2x2 region in the input, the max pooling operation will select the maximum value from that region, effectively reducing the size of the feature map.
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(50, activation='relu'))

model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))   # 3 classes

model.summary()

### Total params: 6,556,515 is the total no. of parameters which includes both weights and bias in our model.
### *Model Complexity: The number of parameters indicates how complex our model is. A model with more parameters has greater capacity to learn intricate patterns, but it also requires more data and computational power to train effectively.

### *Overfitting Risk: A large number of parameters can increase the risk of overfitting, especially if the dataset is small. In this case, our model might "memorize" the training data instead of learning generalizable patterns.

# compile the model

In [ ]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])


#categorical_crossentropy -- calc the error

# Train the model

In [ ]:
cwd = os.getcwd()  # Get current working directory
print("Current Working Directory:", cwd)

In [ ]:
from keras.callbacks import ModelCheckpoint

# train the model
checkpointer = ModelCheckpoint(filepath='saved_model/model.weights.best.keras', save_best_only=True)

hist = model.fit(X_train, Y_train, batch_size=10, epochs=30,
        validation_data=(X_val, Y_val),callbacks=[checkpointer],verbose=1,
           shuffle=True)
#epoch --the number times that the learning algorithm will work through the entire training dataset
#batch_size -- how many samples for each iteration, When the batch size is set to 32, it means that during 
#               each iteration of training, the neural network processes 32 examples (data points) at a time.

In [ ]:
# get predictions on the test set
y_hat = model.predict(X_test)
print(y_hat)


In [ ]:
# evaluate test accuracy
score = model.evaluate(X_test, Y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)
print(accuracy)

In [ ]:
# evaluate training accuracy
score_train = model.evaluate(X_train, Y_train, verbose=0)
accuracy_train = 100 * score_train[1]

# print training accuracy
print('Training accuracy: %.4f%%' % accuracy_train)

#### so the model is overfitting here. Let us apply data augmentation on the training dataset.

Data augmentation is a technique of artificially increasing the training set by creating modified copies of a dataset using existing data.

Data augmentation is particularly useful here since we have:

limited amount of data.
To reduce overfitting.
to increase the diversity of your dataset to avoid overfitting.
to simulate different real-world scenarios (such as varying lighting, orientation, and zoom).


Augmentation techniques:

rotation_range: Randomly rotate the images by up to the specified degrees.
width_shift_range and height_shift_range: Randomly shift the image horizontally or vertically by up to the given fraction of the image's width or height.
shear_range: Applies a shear transformation to images.
zoom_range: Randomly zooms into images.
horizontal_flip: Flips images horizontally with a 50% chance.
fill_mode: Defines how to fill pixels that are shifted or rotated (e.g., using 'nearest' or 'constant').

flow() Method: The flow() method will return batches of augmented images and labels in each iteration. The argument batch_size defines the number of images to return per batch.

Training with Augmented Data: You can pass the train_generator directly to model.fit() to train your model. The generator will provide augmented data on the fly during training.


In [ ]:
# Check the shape of X_train, X_val, and X_test
print('Shape of X_train:', X_train.shape)
print('Shape of X_val:', X_val.shape)
print('Shape of X_test:', X_test.shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Assuming X is your image data (shape: [num_samples, 512, 512, 3])
# and Y is your one-hot encoded labels (shape: [num_samples, 3])

# Step 1: Create the ImageDataGenerator with augmentations
train_datagen = ImageDataGenerator(
    rotation_range=20,        # Random rotations from -20 to 20 degrees
    width_shift_range=0.2,    # Random horizontal shifts
    height_shift_range=0.2,   # Random vertical shifts
    shear_range=0.2,          # Random shearing transformation(tilting the image across the axis)
    zoom_range=0.2,           # Random zoom
    horizontal_flip=True,     # Random horizontal flip(mirror image)
    fill_mode='nearest' ,  # How to fill pixels after transformations
    
)


# Step 3: Fit the generators on the data 
train_datagen.fit(X_train)

In [ ]:

train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Step 5: Train the model using the generators
model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=30,
    validation_data=(X_val, Y_val), # Directly pass validation data
    callbacks=[early_stopping],
    verbose=1
)


In [ ]:
# Step 6: Evaluate the model's performance on test data
score_test = model.evaluate(X_test, Y_test, verbose=2)
accuracy_test = 100 * score_test[1]
print('Test accuracy: %.4f%%' % accuracy_test)

In [ ]:
y_hat = model.predict(X_test)
print(y_hat)

In [ ]:
# evaluate training accuracy
score_train = model.evaluate(X_train, Y_train, verbose=0)
accuracy_train = 100 * score_train[1]

# print training accuracy
print('Training accuracy: %.4f%%' % accuracy_train)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# If y_true and y_pred are one-hot encoded or probabilities, convert to class labels
Y_test_class = np.argmax(Y_test, axis=1)  # Convert one-hot encoded true labels
y_pred_class = np.argmax(y_hat, axis=1)  # Convert predicted probabilities to class labels

# Now you can use classification_report
print(classification_report(Y_test_class, y_pred_class))
print(confusion_matrix(Y_test_class, y_pred_class))

In [ ]:
import tensorflow as tf
img = keras.preprocessing.image.load_img(
    r"C:\Users\new\Downloads\5426924lgpt.jpg", target_size=(512,512,3) # passing new googled image of brownspot disease.
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)

score = predictions[0]
score


plotting all the predicted Y

In [ ]:
fig = plt.figure(figsize=(40, 13))
for i, idx in enumerate(range(X_test.shape[0])):  # Iterate over all test images
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_test[idx]))
    pred_idx = np.argmax(y_hat[idx])
    true_idx = np.argmax(Y_test[idx])
    ax.set_title("{} ({})".format(CATEGORIES[pred_idx], CATEGORIES[true_idx]),
                 color=("blue" if pred_idx == true_idx else "red"))

# Report on Rice Leaf Disease Detection Using Data Augmentation and Model Optimization Techniques

Data augmentation is an essential technique in deep learning, especially when working with a limited dataset. By artificially increasing the size of the training set, augmentation helps the model generalize better, improving its performance on unseen data.

Given the limited size of the dataset (119 images), we explored several techniques, including Data Augmentation, to enhance model performance and avoid overfitting.

### Techniques applied
--------------------------
#### Rotation:
First I rotated the image by 10 degrees, but it was not reducing the overfitting that much.
Then I tried 20 degrees.That not only increased the testing accuracy but also reduced the overfitting.

#### Shearing:
Random shearing transformations that tilt the images along the X or Y axis.
Helps the model recognize leaves under different perspectives.
first I started with giving value o.1. but changing it to 0.2 improved the accuracy

#### Zoom:
Random zooming in or out of the images.
Mimics varying distances and focal lengths.

#### Horizontal Flip:
Randomly flipping images horizontally.
Ensures the model does not become biased by orientation.

#### Fill Mode:
The nearest fill mode was used to fill in missing pixel areas after transformations.

### Impact of Augmentation
------------------------------

I trained two models for comparison:

Model 1: Trained on the original dataset without augmentation.
Model 2: Trained on the dataset with augmentation techniques applied.

Model 1 (No Augmentation):
Training Accuracy: 100%
Test Accuracy: 79.1%

Model 2 (With Augmentation):
Training Accuracy: 98%
Test Accuracy: 87.5%

The performance improvement from 79.1% to 87.5% test accuracy suggests that data augmentation helped the model generalize better, making it less prone to overfitting.

### plotting the augmented images of an image taken from the dataset

In [ ]:
# Load the original image
from tensorflow.keras.preprocessing.image import  load_img,img_to_array
img_path = r'C:\Users\new\Downloads\PRCP-1001-RiceLeaf\data1\Brown spot\DSC_0100.jpg'  # Provide the path to your image
img = load_img(img_path)  # Load the image as PIL format

# Convert image to numpy array
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension



# Create an iterator for generating augmented images
augmented_images = train_datagen.flow(img_array, batch_size=1)

# Plot the original image and augmented images
fig, ax = plt.subplots(1, 7, figsize=(20, 5))

# Display the original image
ax[0].imshow(img)
ax[0].set_title('Original Image')
ax[0].axis('off')

# Display 4 augmented images
for i in range(1, 7):
    augmented_image = next(augmented_images)[0].astype('uint8')  # Generate augmented image
    ax[i].imshow(augmented_image)
    ax[i].set_title(f'Augmented Image {i}')
    ax[i].axis('off')

plt.show()


###  Additional Techniques
----------------------------------

#### Image Preprocessing:
I normalized the pixel values to the range [0, 1] for consistency and to ensure faster convergence during training. Image normalization helps models train faster and more efficiently by scaling the input data.

#### Model Optimization:

Batch Size: I experimented with different batch sizes (32 and 50) to determine the best balance between computational efficiency and model accuracy. A batch size of 32 was optimal for this dataset.

Epochs: I trained the model for 30 epochs, which allowed it to converge while avoiding overfitting.

Early Stopping: Had to apply early stopping during the training process. This technique helps in halting the training when the validation loss stops improving, preventing unnecessary overfitting. By monitoring the validation loss, early stopping ensures that the model does not continue to learn from noise in the training data once the model's performance on the validation set starts to degrade, improving generalization on unseen data.

## challenges faced in this project:

### Limited no. of Images in the dataset:
With only 119 images in the dataset we cant train the deep learning model effectively. In deep learning, the model requires large no. of images in the dataset. And we resolved issue by applying data augmentation.

### Images were of different size:
most deep learning models require input images to have a consistent shape and dimension for efficient processing. To handle this, we did resizing technique to resize it to same size and shape( here we converted it into 512x512x3) before passing it to the model. And we normalized the images by dividing it by 255.

### overfitting:
model without augmentation was overfitting and we reduced it by applying augmentation . so the model with augmentation was the better model to use for production.